In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 51 kB/s 
     |████████████████████████████████| 199 kB 64.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ec30a5fb93fb126dbb4bc18aa15122397e1b5e41b7d2789c4de57b93bd4b2f99
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


Questions
1. Find all the tweets by user
2. Find how many tweets each user has
3. Find all the persons mentioned on tweets
4. Count how many times each person is mentioned
5. Find the 10 most mentioned persons
6. Find all the hashtags mentioned on a tweet
7. Count how many times each hashtag is mentioned
8. Find the 10 most popular Hashtags
9. Find the top 5 countries which tweet the most.

In [2]:
from pyspark.sql import SparkSession , functions as F
spark = SparkSession.builder.appName('twitter').getOrCreate()

In [3]:
from pyspark.sql.types import ArrayType, StringType

In [4]:
df_twitter = spark.read.format('json').load('sample_data/tweets.json')
df_twitter.printSchema()
df_twitter.show(5)

root
 |-- country: string (nullable = true)
 |-- id: string (nullable = true)
 |-- place: string (nullable = true)
 |-- text: string (nullable = true)
 |-- user: string (nullable = true)

+-------------+------------------+---------+--------------------+------------------+
|      country|                id|    place|                text|              user|
+-------------+------------------+---------+--------------------+------------------+
|        India|572692378957430785|   Orissa|@always_nidhi @Yo...|   Srkian_nishu :)|
|United States|572575240615796737|Manhattan|@OnlyDancers Bell...|TagineDiningGlobal|
|United States|572575243883036672|Claremont|1/ "Without the a...|       Daniel Beer|
|United States|572575252020109313|   Vienna|idk why people ha...|  someone actually|
|United States|572575274539356160|   Boston|Taste of Iceland!...|    BostonAttitude|
+-------------+------------------+---------+--------------------+------------------+
only showing top 5 rows



In [27]:
df_twitter.toPandas().to_csv('Output/twitter/twitter_data',index= False)

##1. Find all the tweets by user

In [5]:
user = 'BostonAttitude'
#user = input('Enter user name to see their tweets: ')
question1=df_twitter.filter(df_twitter['user']==user)
question1.show()

+-------------+------------------+------+--------------------+--------------+
|      country|                id| place|                text|          user|
+-------------+------------------+------+--------------------+--------------+
|United States|572575274539356160|Boston|Taste of Iceland!...|BostonAttitude|
+-------------+------------------+------+--------------------+--------------+



In [6]:
question1.toPandas().to_csv('Output/twitter/question1',index= False)

##2. Find how many tweets each user has

In [7]:
question2=df_twitter.groupBy('user').count().orderBy('count', ascending=False)
question2.show(5)

+--------------------+-----+
|                user|count|
+--------------------+-----+
|       #QuissyUpSoon|  258|
|Inès Mendes Askiip ♥|  185|
|           #4Rentinc|  100|
|                  MV|   58|
|    williampriceking|   46|
+--------------------+-----+
only showing top 5 rows



In [9]:
question2.toPandas().to_csv('Output/twitter/question2',index= False)

##3.Find all the persons mentioned on tweets

In [11]:
def generate_usermentioned(text):
    return [item.lstrip('@') for item in text.split(' ') if item.startswith('@')]
user_mentioned = df_twitter.withColumn('users_mentioned', F.udf(lambda text: generate_usermentioned(text), ArrayType(StringType()))('text'))
user_mentioned.show()

+--------------------+------------------+-----------------+--------------------+-------------------+--------------------+
|             country|                id|            place|                text|               user|     users_mentioned|
+--------------------+------------------+-----------------+--------------------+-------------------+--------------------+
|               India|572692378957430785|           Orissa|@always_nidhi @Yo...|    Srkian_nishu :)|[always_nidhi, Yo...|
|       United States|572575240615796737|        Manhattan|@OnlyDancers Bell...| TagineDiningGlobal|       [OnlyDancers]|
|       United States|572575243883036672|        Claremont|1/ "Without the a...|        Daniel Beer|                  []|
|       United States|572575252020109313|           Vienna|idk why people ha...|   someone actually|                  []|
|       United States|572575274539356160|           Boston|Taste of Iceland!...|     BostonAttitude|    [IcelandNatural]|
|       United States|57

In [13]:
user_mentioned.toPandas().to_csv('Output/twitter/question3',index= False)

##4. Count how many times each person is mentioned

In [14]:
new_usermentioned_df= user_mentioned.select(F.explode('users_mentioned').alias('users_mentioned'))
new_usermentioned_df = new_usermentioned_df.filter(new_usermentioned_df['users_mentioned'] != '')
question4=new_usermentioned_df.groupBy('users_mentioned').count()
question4.show(truncate=False)

+---------------+-----+
|users_mentioned|count|
+---------------+-----+
|DjRockyUg      |1    |
|TrillHD        |1    |
|TimmysWell     |1    |
|brookie_baldwin|1    |
|TTTorrez       |2    |
|boytoyjesse    |1    |
|misstoriblack  |1    |
|globalstatmusic|1    |
|_fuckgio       |1    |
|PedroIvoChianca|1    |
|Cpiepz         |1    |
|avachristy3    |1    |
|lostbayouramble|1    |
|bellahadid     |1    |
|sawano_nZk's   |1    |
|marIboros      |1    |
|kochamjacksona |1    |
|WIOD           |2    |
|ShaelynCherie  |2    |
|KevinAnex      |1    |
+---------------+-----+
only showing top 20 rows



In [15]:
question4.toPandas().to_csv('Output/twitter/question4',index= False)

##5. Find the 10 most mentioned persons


In [16]:
question5=new_usermentioned_df.groupBy('users_mentioned').count().orderBy('count', ascending=False).limit(10)
question5.show()

+---------------+-----+
|users_mentioned|count|
+---------------+-----+
|    ShawnMendes|  189|
|  HIITMANonDECK|  100|
|officialdjjuice|   59|
|         MAEJOR|   45|
|    MR_JAYJONES|   41|
|       MeekMill|   35|
|MadisonElleBeer|   30|
|              …|   28|
|     DjLordDash|   27|
|     NICKIMINAJ|   25|
+---------------+-----+



In [17]:
question5.toPandas().to_csv('Output/twitter/question5',index= False)

##6. Find all the hashtags mentioned on a tweet

In [18]:
def generate_hashtags(text):
    return [item for item in text.split(' ') if item.startswith('#')]
hashtags_mentioned = df_twitter.withColumn('hashtags_mentioned', F.udf(lambda text: generate_hashtags(text), ArrayType(StringType()))('text'))
hashtags_mentioned.show()

+--------------------+------------------+-----------------+--------------------+-------------------+--------------------+
|             country|                id|            place|                text|               user|  hashtags_mentioned|
+--------------------+------------------+-----------------+--------------------+-------------------+--------------------+
|               India|572692378957430785|           Orissa|@always_nidhi @Yo...|    Srkian_nishu :)|                  []|
|       United States|572575240615796737|        Manhattan|@OnlyDancers Bell...| TagineDiningGlobal|                  []|
|       United States|572575243883036672|        Claremont|1/ "Without the a...|        Daniel Beer|                  []|
|       United States|572575252020109313|           Vienna|idk why people ha...|   someone actually|                  []|
|       United States|572575274539356160|           Boston|Taste of Iceland!...|     BostonAttitude|                  []|
|       United States|57

In [20]:
hashtags_mentioned.toPandas().to_csv('Output/twitter/question6',index= False)

##7. Count how many times each hashtag is mentioned

In [21]:
new_hashtags_mentioned= hashtags_mentioned.select(F.explode('hashtags_mentioned').alias('hashtags_mentioned'))
new_hashtags_mentioned = new_hashtags_mentioned.filter(new_hashtags_mentioned['hashtags_mentioned'] != '')
question7=new_hashtags_mentioned.groupBy('hashtags_mentioned').count()
question7.show(truncate=False)

+--------------------+-----+
|hashtags_mentioned  |count|
+--------------------+-----+
|#2NE1               |3    |
|#musicLover         |1    |
|#IBMCloud           |2    |
|#flexrecordingstudio|1    |
|#Hottest            |1    |
|#VanessaBorn        |1    |
|#happychappy        |1    |
|#yyjevents          |1    |
|#LittleLionMan      |1    |
|#MBAMBADU           |7    |
|#misheardlyrics     |1    |
|#Indie              |2    |
|#family             |1    |
|#beautiful          |2    |
|#Waiter             |1    |
|#friend             |1    |
|#recuseimitaçoes    |1    |
|#airbnb             |1    |
|#BØRNS              |1    |
|#ChickCorea         |1    |
+--------------------+-----+
only showing top 20 rows



In [22]:
question7.toPandas().to_csv('Output/twitter/question7',index= False)

##8. Find the 10 most popular Hashtags

In [23]:
question8=new_hashtags_mentioned.groupBy('hashtags_mentioned').count().orderBy('count', ascending=False).limit(10)
question8.show()

+-------------------+-----+
| hashtags_mentioned|count|
+-------------------+-----+
|               #DME|  253|
|          #ROADBOYZ|  251|
|             #music|  236|
|             #Paris|  144|
|#QuissyUpSoon🔥🔥💯|  129|
|      #QuissyUpSoon|  120|
| #Trippythursdaymia|  100|
|             #Music|   84|
|    #MaejorMeAndYou|   44|
|              #IGGL|   41|
+-------------------+-----+



In [24]:
question8.toPandas().to_csv('Output/twitter/question8',index= False)

##9.Find the top 5 countries which tweet the most.

In [25]:
question9=df_twitter.groupBy('country').count().orderBy('count', ascending=False).limit(5)
question9.show()

+--------------+-----+
|       country|count|
+--------------+-----+
| United States| 4841|
|        France|  737|
|     Indonesia|  370|
|United Kingdom|  365|
|        Brasil|  256|
+--------------+-----+



In [26]:
question9.toPandas().to_csv('Output/twitter/question9',index= False)